## 1. 환경 설정 및 임포트

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import asyncio
import logging
from pathlib import Path
from dotenv import load_dotenv
from helper_dev_utils import get_auto_logger
logger = get_auto_logger()

# 프로젝트 루트 경로 추가
project_root = Path.cwd().parent.parent
src_path = project_root / "src"

# sys.path에 추가
for path in [str(project_root), str(src_path)]:
    if path not in sys.path:
        sys.path.insert(0, path)
        logger.debug(f"path: {path}")

logger.debug(f"프로젝트 루트: {project_root}")
logger.debug(f"소스 경로: {src_path}")

# 환경 변수 로드
env_path = project_root / ".env"
if env_path.exists():
    load_dotenv(env_path)
    logger.debug(f"환경 변수 로드: {env_path}")
else:
    logger.warning(f".env 파일을 찾을 수 없습니다 ({env_path})")

2026-01-09 14:15:10 W [helper_utils_colab:211] - GoogleDrive not found, falling back to temp folder: /tmp
2026-01-09 14:15:10 W [helper_utils_colab:122] - Cache directory not found, falling back to temp folder: /tmp
2026-01-09 14:15:10 D [ipykernel_launcher:18] - path: /home/spai0433/codeit-ai-3team-ad-content
2026-01-09 14:15:10 D [ipykernel_launcher:18] - path: /home/spai0433/codeit-ai-3team-ad-content/src
2026-01-09 14:15:10 D [ipykernel_launcher:20] - 프로젝트 루트: /home/spai0433/codeit-ai-3team-ad-content
2026-01-09 14:15:10 D [ipykernel_launcher:21] - 소스 경로: /home/spai0433/codeit-ai-3team-ad-content/src
2026-01-09 14:15:10 D [ipykernel_launcher:27] - 환경 변수 로드: /home/spai0433/codeit-ai-3team-ad-content/.env


## 2. OpenAI API 키 확인

In [3]:
# OpenAI API 키 확인
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    logger.debug("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
    logger.debug("   .env 파일에 다음을 추가하세요:")
    logger.debug("   OPENAI_API_KEY=sk-...")
else:
    logger.debug(f"OpenAI API 키 확인됨 (길이: {len(openai_api_key)} 문자)")
    logger.debug(f"   키 prefix: {openai_api_key[:5]} --- {openai_api_key[-5:]}")

2026-01-09 14:15:10 D [ipykernel_launcher:9] - OpenAI API 키 확인됨 (길이: 164 문자)
2026-01-09 14:15:10 D [ipykernel_launcher:10] -    키 prefix: sk-pr --- cxzEA


## 3. mcpadapter 임포트 및 초기화

In [4]:
# mcpadapter 임포트
from mcpadapter import MCPClient, LLMAdapter

# 설정
MCP_SERVER_URL = "http://localhost:3000"

logger.info("mcpadapter 임포트 완료")
logger.info(f"   MCP 서버 URL: {MCP_SERVER_URL}")

2026-01-09 14:15:24 I [ipykernel_launcher:7] - mcpadapter 임포트 완료
2026-01-09 14:15:24 I [ipykernel_launcher:8] -    MCP 서버 URL: http://localhost:3000


## 4. 이미지 경로 설정

In [5]:
# 이미지 파일 경로 설정
script_dir = Path.cwd()
product_image_path = script_dir / "image.png"
output_dir = script_dir / "outputs"
output_dir.mkdir(exist_ok=True)

# 출력 파일 경로
banana_output_path = output_dir / "banana_ad_result.png"

# 이미지 존재 확인
if product_image_path.exists():
    logger.info(f"제품 이미지 확인: {product_image_path}")
else:
    logger.warning(f"제품 이미지 없음: {product_image_path}")
    
logger.info(f"   출력 경로: {banana_output_path}")

2026-01-09 14:15:31 I [ipykernel_launcher:12] - 제품 이미지 확인: /home/spai0433/codeit-ai-3team-ad-content/script/김명환/image.png
2026-01-09 14:15:31 I [ipykernel_launcher:16] -    출력 경로: /home/spai0433/codeit-ai-3team-ad-content/script/김명환/outputs/banana_ad_result.png


## 4-1. 서버 상태 대기 헬퍼 함수

In [6]:
async def wait_for_server_ready(max_wait_sec: int = 300, check_interval: int = 10) -> bool:
    """
    AI 서버가 idle 상태가 될 때까지 대기
    
    Args:
        max_wait_sec: 최대 대기 시간 (초, 기본값 300초 = 5분)
        check_interval: 상태 확인 간격 (초, 기본값 10초)
    
    Returns:
        True: 서버가 idle 상태
        False: 타임아웃 발생
    """
    logger.info(f"🕐 AI 서버 idle 상태 대기 중... (최대 {max_wait_sec}초)")
    
    async with MCPClient(
        base_url=MCP_SERVER_URL,
        timeout=30
    ) as client:
        elapsed = 0
        while elapsed < max_wait_sec:
            try:
                health = await client.call_tool("check_server_health", {})
                
                # health가 문자열인 경우 파싱
                if isinstance(health, str):
                    # "상태: busy" 또는 "상태: idle" 패턴 찾기
                    if "idle" in health.lower():
                        logger.info(f"   서버 idle 상태 확인 (대기 시간: {elapsed}초)")
                        return True
                    elif "busy" in health.lower():
                        logger.info(f"   ⏳ 서버 busy 상태... {check_interval}초 후 재확인 (경과: {elapsed}초/{max_wait_sec}초)")
                    else:
                        # 상태를 알 수 없으면 바로 진행
                        logger.warning(f"   서버 상태 불명: {health}")
                        return True
                else:
                    # dict 형태인 경우
                    status = health.get("status", "unknown") if isinstance(health, dict) else "unknown"
                    if status == "idle":
                        logger.info(f"   서버 idle 상태 확인 (대기 시간: {elapsed}초)")
                        return True
                    elif status == "busy":
                        logger.info(f"   ⏳ 서버 busy 상태... {check_interval}초 후 재확인 (경과: {elapsed}초/{max_wait_sec}초)")
                    else:
                        logger.warning(f"   서버 상태: {status}")
                        return True
                
            except Exception as e:
                logger.warning(f"   서버 상태 확인 실패: {e}")
                # 상태 확인 실패 시 그냥 진행
                return True
            
            await asyncio.sleep(check_interval)
            elapsed += check_interval
        
        logger.warning(f"   ⏰ 타임아웃: {max_wait_sec}초 동안 서버가 idle 상태가 되지 않음")
        return False

logger.info("서버 대기 헬퍼 함수 정의 완료")

2026-01-09 14:15:38 I [ipykernel_launcher:59] - 서버 대기 헬퍼 함수 정의 완료


## 5. 방법 1: LLMAdapter를 사용한 자연어 기반 광고 생성

LLMAdapter는 자연어로 요청하면 자동으로 적절한 MCP 도구를 선택하여 호출합니다.

In [7]:
# LLMAdapter를 사용한 바나나 광고 생성 (재시도 로직 포함)
async def test_llm_adapter_banana_ad(max_retries: int = 3):
    """
    LLMAdapter를 사용하여 자연어로 바나나 광고 생성 (재시도 로직 포함)
    
    Args:
        max_retries: 최대 재시도 횟수 (기본값: 3)
    """
    logger.info("=" * 60)
    logger.info("방법 1: LLMAdapter로 바나나 광고 생성 시작")
    logger.info("=" * 60)
    
    # 서버 idle 상태 대기
    if not await wait_for_server_ready(max_wait_sec=300):
        logger.warning("서버가 busy 상태이지만 계속 진행합니다.")
    
    # LLMAdapter 초기화
    async with LLMAdapter(
        openai_api_key=openai_api_key,
        mcp_server_url=MCP_SERVER_URL,
        model="gpt-5-mini"
    ) as adapter:
        
        # 자연어로 광고 생성 요청
        user_message = f"""
바나나 제품의 시장 판매용 광고 이미지를 생성해주세요.

제품 정보:
- 제품 이미지 경로: {product_image_path}
- 광고 텍스트: "맛있는바나나 2500원 시장 자판"
- 판매 장소: 전통시장 자판

요구사항:
1. 배경: 활기찬 전통시장 분위기 (밝고 따뜻한 조명)
2. 텍스트 스타일: 굵고 선명한 3D 한글 텍스트, 노란색/금색 그라데이션
3. 적당한 폰트 크기와 위치: 텍스트가 이미지 중앙에 잘 보이도록 배치
4. 출력 이미지 경로: {banana_output_path}

generate_ad_image 도구를 사용하여 광고를 생성하고 결과를 반환해주세요.
"""
        
        # 재시도 로직
        retry_delays = [60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60]
        
        for attempt in range(max_retries):
            try:
                logger.info(f"LLM에 요청 전송 중... (시도 {attempt + 1}/{max_retries})")
                response = await adapter.chat(user_message, max_tool_calls=5)
                
                logger.info("=" * 60)
                logger.info("LLM 응답:")
                logger.info(response)
                logger.info("=" * 60)
                
                # 결과 이미지 확인
                if banana_output_path.exists():
                    logger.info(f"광고 이미지 생성 완료: {banana_output_path}")
                    
                    # 이미지 표시 (Jupyter에서만 작동)
                    try:
                        from IPython.display import Image, display
                        display(Image(filename=str(banana_output_path)))
                    except:
                        logger.info("   (이미지 표시 기능은 Jupyter 환경에서만 작동합니다)")
                    
                    return response
                else:
                    logger.warning(f"출력 이미지를 찾을 수 없습니다: {banana_output_path}")
                    
                    # 이미지가 없어도 응답이 있으면 성공으로 간주
                    if response and "busy" not in response.lower() and "error" not in response.lower():
                        logger.info("   응답은 정상이지만 이미지 파일이 없습니다.")
                        return response
                    
                    # 재시도가 필요한 경우
                    if attempt < max_retries - 1:
                        wait_time = retry_delays[attempt]
                        logger.info(f"   {wait_time}초 후 재시도합니다...")
                        await asyncio.sleep(wait_time)
                    else:
                        logger.error("   모든 재시도 실패")
                        return response

            except Exception as e:
                logger.error(f"시도 {attempt + 1} 실패: {e}")
                
                if attempt < max_retries - 1:
                    wait_time = retry_delays[attempt]
                    logger.info(f"   {wait_time}초 후 재시도합니다...")
                    await asyncio.sleep(wait_time)
                else:
                    logger.error("   모든 재시도 실패")
                    raise
        
        return None


# 비동기 함수 실행
await test_llm_adapter_banana_ad()


2026-01-09 14:15:45 I [ipykernel_launcher:9] - ============================================================
2026-01-09 14:15:45 I [ipykernel_launcher:10] - 방법 1: LLMAdapter로 바나나 광고 생성 시작
2026-01-09 14:15:45 I [ipykernel_launcher:11] - ============================================================
2026-01-09 14:15:45 I [ipykernel_launcher:13] - 🕐 AI 서버 idle 상태 대기 중... (최대 300초)
2026-01-09 14:15:47 I [ipykernel_launcher:31] -    ⏳ 서버 busy 상태... 10초 후 재확인 (경과: 0초/300초)
2026-01-09 14:15:57 I [ipykernel_launcher:31] -    ⏳ 서버 busy 상태... 10초 후 재확인 (경과: 10초/300초)
2026-01-09 14:16:07 I [ipykernel_launcher:31] -    ⏳ 서버 busy 상태... 10초 후 재확인 (경과: 20초/300초)
2026-01-09 14:16:17 I [ipykernel_launcher:31] -    ⏳ 서버 busy 상태... 10초 후 재확인 (경과: 30초/300초)
2026-01-09 14:16:27 I [ipykernel_launcher:31] -    ⏳ 서버 busy 상태... 10초 후 재확인 (경과: 40초/300초)
2026-01-09 14:16:37 I [ipykernel_launcher:31] -    ⏳ 서버 busy 상태... 10초 후 재확인 (경과: 50초/300초)
2026-01-09 14:16:47 I [ipykernel_launcher:31] -    ⏳ 서버 busy 상태... 10초 

'요청해주신 광고 생성 작업을 시작했으나 AI 서버 오류로 생성이 실패했습니다.\n\n발생한 오류:\n- 첫 시도: 연결 에러\n- 두번째 시도: 서버 과부하(서버가 사용 중). 응답에 따르면 573초(약 9.5분) 후 재시도 권장.\n\n제가 전달한 생성 파라미터(요청하신 내용 기준)\n- 제품 이미지: /home/spai0433/codeit-ai-3team-ad-content/script/김명환/image.png\n- 배경 프롬프트: 활기찬 전통시장 분위기(밝고 따뜻한 조명 등)\n- 광고 텍스트: "맛있는바나나 2500원 시장 자판"\n- 텍스트 스타일: 굵고 선명한 3D 한글, 노란색→금색 그라데이션, 광택/음영 등\n- 합성 방식: overlay\n- 텍스트 위치: center\n- 출력 경로: /home/spai0433/codeit-ai-3team-ad-content/script/김명환/outputs/banana_ad_result.png\n\n다음 중 어떻게 진행할지 골라 주세요:\n1) 기다렸다가(권장 대기 시간 약 573초) 동일 요청으로 재시도 — 제가 바로 재시도할게요. (실제 재시도는 즉시 실행하되 서버가 여전히 바쁘면 또 실패할 수 있습니다.)\n2) 테스트(더미) 모드로 즉시 생성 — test_mode=true 옵션을 사용해 빠르게 샘플 이미지를 생성해 드립니다(실제 서버 로드가 낮을 때와 동일 품질은 아닐 수 있음).\n3) 다른 설정(예: composition_mode 변경, text_position 변경, 폰트 지정)을 먼저 하여 재시도.\n4) 지금은 작업 보류 — 나중에 다시 시도하길 원하시면 시간 알려 주세요.\n\n원하시는 옵션 번호(또는 추가 지시)를 알려 주세요. 제가 바로 진행하겠습니다.'

## 6. 방법 2: MCPClient를 사용한 직접 도구 호출

MCPClient를 사용하면 MCP 도구를 직접 호출할 수 있습니다. 파라미터를 명시적으로 제어할 수 있습니다.

In [ ]:
# MCPClient를 사용한 직접 도구 호출 (재시도 로직 포함)
async def test_mcp_client_direct_call(max_retries: int = 3):
    """
    MCPClient를 사용하여 generate_ad_image 도구를 직접 호출 (재시도 로직 포함)
    
    Args:
        max_retries: 최대 재시도 횟수 (기본값: 3)
    """
    logger.info("=" * 60)
    logger.info("방법 2: MCPClient로 직접 도구 호출")
    logger.info("=" * 60)
    
    # 서버 idle 상태 대기
    if not await wait_for_server_ready(max_wait_sec=300):
        logger.warning("서버가 busy 상태이지만 계속 진행합니다.")
    
    try:
        # MCPClient 초기화 (타임아웃 증가: 10분 → 15분)
        async with MCPClient(
            base_url=MCP_SERVER_URL,
            timeout=900  # 15분
        ) as client:
            
            # 서버 상태 확인
            logger.info("🔍 MCP 서버 상태 확인 중...")
            try:
                health = await client.call_tool("check_server_health", {})
                logger.info(f"   MCP 서버 정상:")
                logger.info(f"   {health}")
            except Exception as e:
                logger.warning(f"   서버 상태 확인 실패 (계속 진행): {e}")
            
            # 사용 가능한 폰트 목록 확인
            logger.info("🔍 사용 가능한 폰트 확인 중...")
            try:
                fonts = await client.call_tool("list_available_fonts", {})
                logger.info(f"   사용 가능한 폰트: {fonts}")
            except Exception as e:
                logger.warning(f"   폰트 목록 확인 실패 (계속 진행): {e}")
            
            # 광고 이미지 생성 파라미터
            params = {
                "product_image_path": str(product_image_path),
                "background_prompt": "Vibrant traditional Korean market scene, colorful fruit stalls, warm lighting, bustling atmosphere",
                "text_content": "맛있는바나나 2500원 시장 자판",
                "text_style_prompt": "Bold 3D Korean text with bright yellow and gold gradient, glossy metallic surface, eye-catching",
                "font_name": "NanumGothicBold.ttf",
                "composition_mode": "overlay",
                "text_position": "center",
                "test_mode": True,  # 빠른 테스트용
                "wait_for_completion": True,  # 결과를 즉시 받음
                "save_output_path": str(banana_output_path)
            }
            
            # 재시도 로직
            retry_delays = [30, 60, 120]  # 재시도 대기 시간 (초)
            result = None
            
            for attempt in range(max_retries):
                try:
                    logger.info(f"generate_ad_image 도구 호출 중... (시도 {attempt + 1}/{max_retries})")
                    logger.info(f"   파라미터:")
                    for key, value in params.items():
                        logger.info(f"      {key}: {value}")
                    
                    # 도구 호출
                    result = await client.call_tool("generate_ad_image", params)
                    
                    logger.info("=" * 60)
                    logger.info("도구 호출 결과:")
                    logger.info(result)
                    logger.info("=" * 60)
                    
                    # 결과 이미지 확인
                    if banana_output_path.exists():
                        logger.info(f"광고 이미지 생성 완료: {banana_output_path}")
                        
                        # 이미지 표시
                        try:
                            from IPython.display import Image, display
                            display(Image(filename=str(banana_output_path)))
                        except:
                            logger.info("   (이미지 표시 기능은 Jupyter 환경에서만 작동합니다)")
                        
                        return result
                    else:
                        logger.warning(f"출력 이미지를 찾을 수 없습니다: {banana_output_path}")
                        
                        # 결과 메시지에 "완료" 또는 성공 관련 키워드가 있으면 성공으로 간주
                        if result and isinstance(result, str):
                            if "완료" in result or "success" in result.lower():
                                logger.info("   응답은 정상이지만 이미지 파일이 없습니다.")
                                return result
                            elif "busy" in result.lower() or "503" in result:
                                logger.warning(f"   서버 busy 상태 감지")
                                if attempt < max_retries - 1:
                                    wait_time = retry_delays[attempt]
                                    logger.info(f"   {wait_time}초 후 재시도합니다...")
                                    await asyncio.sleep(wait_time)
                                    continue
                        
                        # 마지막 시도였다면 결과 반환
                        if attempt == max_retries - 1:
                            logger.error("   모든 재시도 후에도 이미지 생성 실패")
                            return result
                            
                except Exception as e:
                    logger.error(f"시도 {attempt + 1} 실패: {e}")
                    
                    # 503 에러인지 확인
                    is_503_error = "503" in str(e) or "Service Unavailable" in str(e) or "busy" in str(e).lower()
                    
                    if is_503_error and attempt < max_retries - 1:
                        wait_time = retry_delays[attempt]
                        logger.info(f"   서버 busy 상태 - {wait_time}초 후 재시도합니다...")
                        await asyncio.sleep(wait_time)
                    elif attempt == max_retries - 1:
                        logger.error("   모든 재시도 실패")
                        import traceback
                        logger.error(traceback.format_exc())
                        raise
                    else:
                        # 503이 아닌 다른 에러는 즉시 종료
                        logger.error("   재시도 불가능한 에러 발생")
                        import traceback
                        logger.error(traceback.format_exc())
                        raise
            
            return result
            
    except Exception as e:
        logger.error(f"MCPClient 테스트 실패: {e}")
        import traceback
        logger.error(traceback.format_exc())
        raise

# 비동기 함수 실행
await test_mcp_client_direct_call()


## 7. 비동기 작업 패턴: Job ID 기반 폴링

대용량 작업의 경우 `wait_for_completion=False`로 설정하여 즉시 job_id를 받고, 상태를 주기적으로 확인할 수 있습니다.

In [ ]:
# 비동기 작업 패턴: Job ID 기반 폴링
async def test_async_job_pattern():
    """
    비동기 작업 패턴: Job ID를 받아서 상태 확인하는 방식
    """
    logger.info("=" * 60)
    logger.info("방법 3: 비동기 작업 패턴 (Job ID 폴링)")
    logger.info("=" * 60)
    
    try:
        async with MCPClient(
            base_url=MCP_SERVER_URL,
            timeout=600
        ) as client:
            
            # Step 1: 광고 생성 시작 (즉시 job_id 반환)
            params = {
                "product_image_path": str(product_image_path),
                "background_prompt": "Vibrant traditional Korean market scene, colorful fruit stalls, warm lighting",
                "text_content": "맛있는바나나 2500원 시장 자판",
                "text_style_prompt": "Bold 3D Korean text with yellow-gold gradient",
                "font_name": "NanumGothicBold.ttf",
                "composition_mode": "overlay",
                "text_position": "center",
                "test_mode": True,
                "wait_for_completion": False,  # 즉시 job_id 반환
                "save_output_path": str(banana_output_path)
            }
            
            logger.info("Step 1: 광고 생성 작업 시작...")
            start_result = await client.call_tool("generate_ad_image", params)
            logger.info(f"Job 시작됨: {start_result}")
            
            # job_id 추출 (실제 구현에 따라 다를 수 있음)
            job_id = start_result.get("job_id") if isinstance(start_result, dict) else None
            
            if not job_id:
                logger.warning("job_id를 찾을 수 없습니다. 동기 방식으로 처리된 것 같습니다.")
                return start_result
            
            logger.info(f"   Job ID: {job_id}")
            
            # Step 2: 상태 폴링 (10초마다)
            logger.info("📊 Step 2: 작업 상태 확인 중...")
            max_attempts = 30  # 최대 5분 (30 * 10초)
            attempt = 0
            
            while attempt < max_attempts:
                await asyncio.sleep(10)
                attempt += 1
                
                # 상태 확인
                status_result = await client.call_tool(
                    "check_generation_status",
                    {"job_id": job_id}
                )
                
                logger.info(f"   [{attempt}/{max_attempts}] 상태: {status_result}")
                
                # 완료 여부 확인 (실제 구현에 따라 다를 수 있음)
                if isinstance(status_result, dict):
                    status = status_result.get("status")
                    progress = status_result.get("progress", 0)
                    
                    if status == "completed" and progress >= 100:
                        logger.info("작업 완료!")
                        break
                    elif status == "failed":
                        logger.error(f"작업 실패: {status_result.get('error')}")
                        return status_result
            
            # Step 3: 결과 확인
            if banana_output_path.exists():
                logger.info(f"광고 이미지 생성 완료: {banana_output_path}")
                
                try:
                    from IPython.display import Image, display
                    display(Image(filename=str(banana_output_path)))
                except:
                    logger.info("   (이미지 표시 기능은 Jupyter 환경에서만 작동합니다)")
            else:
                logger.warning(f"출력 이미지를 찾을 수 없습니다: {banana_output_path}")
            
            return status_result
            
    except Exception as e:
        logger.error(f"비동기 작업 패턴 테스트 실패: {e}")
        import traceback
        logger.error(traceback.format_exc())
        raise

# 주석 처리 (필요시 실행)
# await test_async_job_pattern()

## 8. 사용 가이드 및 참고사항

### MCP 서버 실행 방법

테스트를 실행하기 전에 MCP 서버를 먼저 시작해야 합니다:

```bash
# 방법 1: Docker Compose로 실행
cd /home/spai0433/codeit-ai-3team-ad-content/src/nanoCocoa_mcpserver
docker-compose up -d

# 방법 2: 직접 실행
python -m nanoCocoa_mcpserver.server
```

### 테스트 모드 vs 프로덕션 모드

- **test_mode=True**: 빠른 테스트용 (1-3초), 더미 이미지 생성
- **test_mode=False**: 실제 AI 생성 (30-120초), 고품질 이미지

### 세 가지 사용 패턴 비교

| 방법 | 장점 | 단점 | 사용 시기 |
|------|------|------|----------|
| **LLMAdapter** | 자연어로 간편하게 요청 | OpenAI API 비용 발생 | 프론트엔드 챗봇, 유연한 UI |
| **MCPClient (동기)** | 파라미터 명시적 제어, 빠른 응답 | 긴 작업 시 블로킹 | 단순 API, 빠른 작업 |
| **MCPClient (비동기)** | 긴 작업도 논블로킹, 진행률 추적 | 구현 복잡도 증가 | 대용량 작업, 백그라운드 처리 |

### 한글 폰트 사용

한글 텍스트를 렌더링하려면 한글 폰트가 필수입니다:

- `NanumGothic.ttf` - 나눔고딕 (기본)
- `NanumGothicBold.ttf` - 나눔고딕 볼드
- `NanumMyeongjo.ttf` - 나눔명조

폰트 목록 확인:
```python
fonts = await client.call_tool("list_available_fonts", {})
```

### 재시도 로직

방법 1, 2 함수는 AI 서버 busy 상태(503 에러)를 자동으로 처리합니다:

- **자동 재시도**: 최대 3회 (기본값)
- **재시도 간격**: 30초 → 60초 → 120초 (고정 간격)
- **서버 대기**: 각 함수 시작 전 AI 서버가 idle 상태가 될 때까지 최대 5분 대기
- **타임아웃**: MCPClient 타임아웃 15분 (기존 10분에서 증가)

### 트러블슈팅

1. **연결 오류**: MCP 서버가 실행 중인지 확인 (`http://localhost:3000`)
2. **이미지 경로 오류**: 절대 경로 사용 권장
3. **한글 깨짐**: 한글 폰트 지정 필수
4. **메모리 부족**: test_mode=True로 먼저 테스트
5. **서버 busy 오류 (503)**:
   - 자동 재시도 로직이 작동합니다 (최대 3회)
   - 수동으로 대기하려면: `await wait_for_server_ready()` 함수 사용
   - 서버 상태 확인: `await client.call_tool("check_server_health", {})`
6. **이미지가 생성되지 않음**:
   - 응답 메시지에 "완료"가 있는지 확인
   - AI 서버가 실제로 작업을 완료했는지 로그 확인
   - `save_output_path` 경로 권한 확인

---

## 실행 순서

1. **환경 설정**: 셀 1-2 실행 (환경 변수 및 OpenAI API 키 확인)
2. **mcpadapter 초기화**: 셀 3-4 실행 (임포트 및 이미지 경로 설정)
3. **테스트 실행**:
   - **방법 1 (LLMAdapter)**: 셀 5 실행 - 자연어 기반 광고 생성
   - **방법 2 (MCPClient 동기)**: 셀 6 실행 - 직접 도구 호출
   - **방법 3 (MCPClient 비동기)**: 셀 7 실행 - Job ID 폴링 패턴

각 방법은 독립적으로 실행 가능합니다!

In [ ]:
# 간단한 테스트: 서버 상태 확인
async def quick_test():
    """MCP 서버 연결 및 상태 확인"""
    logger.info("=== 빠른 연결 테스트 ===")
    
    async with MCPClient(
        base_url=MCP_SERVER_URL,
        timeout=30
    ) as client:
        # 서버 상태 확인
        health = await client.call_tool("check_server_health", {})
        logger.info(f"서버 상태:\n{health}")
        
        # 폰트 목록 확인
        fonts = await client.call_tool("list_available_fonts", {})
        logger.info(f"사용 가능한 폰트:\n{fonts}")
        
    logger.info("=== 테스트 완료 ===")

await quick_test()
